In [ ]:
OLLAMA_BASE_URL = "http://ollama:11434/v1"
MODEL = "llama3.2"

In [7]:
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.ollama import OllamaProvider


ollama_model = OpenAIChatModel(
    model_name=MODEL,
    provider=OllamaProvider(base_url=OLLAMA_BASE_URL),
)

In [ ]:
import functools
import json


def log_function_call(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with: {kwargs}")
        result = func(*args, **kwargs)
        print(f"Function {func.__name__} returned: {json.dumps(result, indent=2)}")
        return result

    return wrapper


## Function Calling (Tools)

Function calling allows the LLM to use external tools. You can define a set of functions and the model will intelligently decide when to use them based on the user's prompt. This is a powerful way to extend the capabilities of the model, allowing it to interact with external APIs, databases, or any other system.

Let's create a simple calculator tool and see how the agent can use it.


In [ ]:
import requests
import time
from pydantic_ai import Agent


@log_function_call
def get_weather(latitude: float, longitude: float) -> dict:
    """Get current weather data for a specific location."""
    url = (
        "https://api.open-meteo.com/v1/forecast"
        f"?latitude={latitude}&longitude={longitude}"
        "&current=temperature_2m,wind_speed_10m"
    )
    response = requests.get(url)
    return response.json()


agent = Agent(
    ollama_model,
    tools=[get_weather],
    system_prompt=f"You are a helpful assistant that can use tools to get information. Current time is {time.time()}",
)

In [ ]:
result = await agent.run(f"How windy is it in Warsaw today?")
print(result.output, end="\n\n")

Calling get_weather with: {'latitude': 52.2345, 'longitude': 21.0099}
Function get_weather returned: {
  "latitude": 52.23009,
  "longitude": 21.017075,
  "generationtime_ms": 0.034332275390625,
  "utc_offset_seconds": 0,
  "timezone": "GMT",
  "timezone_abbreviation": "GMT",
  "elevation": 114.0,
  "current_units": {
    "time": "iso8601",
    "interval": "seconds",
    "temperature_2m": "\u00b0C",
    "wind_speed_10m": "km/h"
  },
  "current": {
    "time": "2025-10-31T15:30",
    "interval": 900,
    "temperature_2m": 9.7,
    "wind_speed_10m": 10.4
  }
}
According to the current weather conditions, the wind speed in Warsaw is approximately 10.4 km/h as of 3:30 PM local time on October 31st, 2025.



## Saving State from Tools

Sometimes you want to persist data that tools collect during execution. PydanticAI allows you to pass a state object (`deps`) through the agent's context, which tools can read from and write to.

In [ ]:
from pydantic import BaseModel
from pydantic_ai import Agent, RunContext
import requests
from typing import Optional


class WeatherState(BaseModel):
    last_temperature: Optional[float] = None
    last_location: Optional[str] = None


def get_weather_with_state(ctx: RunContext[WeatherState], latitude: float, longitude: float, location_name: str) -> str:
    """Get weather and save the temperature to state."""
    url = (
        "https://api.open-meteo.com/v1/forecast"
        f"?latitude={latitude}&longitude={longitude}"
        "&current=temperature_2m,wind_speed_10m"
    )
    response = requests.get(url)
    data = response.json()
    
    temperature = data["current"]["temperature_2m"]
    
    ctx.deps.last_temperature = temperature
    ctx.deps.last_location = location_name
    
    return f"Temperature in {location_name}: {temperature}°C"


agent = Agent(
    ollama_model,
    deps_type=WeatherState,
    tools=[get_weather_with_state],
    system_prompt="You are a helpful weather assistant."
)

In [ ]:
state = WeatherState()
result = await agent.run(
    "What's the temperature in Warsaw? Coordinates: 52.23, 21.01",
    deps=state
)

print("Agent response:", result.output)
print(f"\nSaved to state:")
print(f"  Location: {state.last_location}")
print(f"  Temperature: {state.last_temperature}°C")


Agent response: The current temperature in Warsaw is approximately 5.9°C (42.4°F).

Saved to state:
  Location: Warsaw
  Temperature: 5.9°C
